<a href="https://colab.research.google.com/github/dp457/LangChainLLM/blob/main/Agent_Design_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1) Install Ollama in the Colab VM
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!apt-get install -y pciutils lshw

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 2s (496 kB/s)
Selecting previously unselected package pc

In [5]:
!nvidia-smi

Sat Aug 30 17:47:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# 2) Start the server in the background
import subprocess, time
server = subprocess.Popen(["ollama", "serve"])
time.sleep(5)  # give it a moment to boot

In [3]:
# 3) Pull a smallish model
!ollama pull llama3.1

In [6]:
!pip install langchain-ollama

In [7]:
from langchain_ollama import ChatOllama
llm = ChatOllama(base_url="http://127.0.0.1:11434", model="llama3.1", temperature=0.3)

from langchain.chat_models import init_chat_model

model = init_chat_model(
    "llama3.1",                # any model you've pulled in Ollama
    model_provider="ollama",
    base_url="http://127.0.0.1:11434",
    temperature=0.3,
)

In [9]:
!pip install langchain-tavily

In [11]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 23.8 MB/s eta 0:00:00


In [12]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [16]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter the Key:\n")

In [15]:
# Create agent
memory = MemorySaver()
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [20]:
# Use the agent
config = {"configurable":{"thread_id":"abc123"}}

input_message = {
 "role": "user",
 "content": "Hi, I am Deepak and I live in Bedford, Bedfordshire, United Kingdom",
}

In [21]:
for step in agent_executor.stream({"messages": [input_message]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I am Deepak and I live in Bedford, Bedfordshire, United Kingdom
================================== Ai Message ==================================
Tool Calls:
  tavily_search (5b0c8cdb-371f-47be-8159-1654af6116f1)
 Call ID: 5b0c8cdb-371f-47be-8159-1654af6116f1
  Args:
    query: Bedford Bedfordshire UK
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "Bedford Bedfordshire UK", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://en.wikipedia.org/wiki/Bedford", "title": "Bedford - Wikipedia", "content": "# Bedford | Bedford | | | Post town | Bedford | **Bedford** is a market town in Bedfordshire, England. Bedford is the county town of Bedfordshire and seat of the Borough of Bedford local government district. Kempston is the only civil parish in both the built-up area and Borough of Bedford

In [23]:
input_message = {
    "role": "user",
    "content": "What's the current weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the current weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (7a430b93-1bef-4a39-a538-6c308f4fc63a)
 Call ID: 7a430b93-1bef-4a39-a538-6c308f4fc63a
  Args:
    query: current weather in Bedford England
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Bedford England", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Bedford, England", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'England', 'region': 'Oppland', 'country': 'Norway', 'lat': 61.2333, 'lon': 10.2, 'tz_id': 'Europe/Oslo', 'localtime_epoch': 1756576700, 'localtime': '2025-08-30 19:58'}, 'current': {'last_updated_epoch': 1756575900, 'last_updated': '2025-08-30 19:45', 'temp_c': 12.1, 'temp_

In [24]:
!pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 12.9 MB/s eta 0:00:00


In [25]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in Bedford, Bedfordshire, United Kingdom")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in Bedford, Bedfordshire, United Kingdom', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in Bedford, Bedfordshire, United Kingdom', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Bedford', 'region': 'Bedfordshire', 'country': 'United Kingdom', 'lat': 52.1337, 'lon': -0.4601, 'tz_id': 'Europe/London', 'localtime_epoch': 1756577484, 'localtime': '2025-08-30 19:11'}, 'current': {'last_updated_epoch': 1756576800, 'last_updated': '2025-08-30 19:00', 'temp_c': 17.6, 'temp_f': 63.6, 'is_day': 1, 'condition': {'text': 'Patchy rain nearby', 'icon': '//cdn.weatherapi.com/weather/64x64/day/176.png', 'code': 1063}, 'wind_mph': 19.9, 'wind_kph': 32.0, 'wind_degree': 189, 'wind_dir': 'S', 'pressure_mb': 999.0, 'pressure_in': 29.5, 'precip_mm': 0.09, 'precip_in': 0.0, 'humidity': 88, 'cloud': 100, 'feelslike_c': 17.6, 'feelslike_f': 63.6, 'windchill_c': 17.6, 'windchill_f': 63.6, 'heatindex_c': 17.6, 

In [26]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

··········


# Agent Building Using Language Models



In [27]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

We can now see what it is like to enable this model to do tool calling. In order to enable that we use `.bind_tools` to give the language model knowledge of these tools

In [28]:
model_with_tools = model.bind_tools(tools)

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the `content` field as well as the `tool_calls` field.

In [29]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'Hi!'}, 'id': 'f19eb8d7-63ab-49c0-b521-e89bf3d23373', 'type': 'tool_call'}]


In [30]:
query = "Search Premier League football Results today"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'Premier League football results today', 'search_depth': 'advanced'}, 'id': 'd64c9805-a5e7-494d-96a1-0e7b462e1e6c', 'type': 'tool_call'}]


As it can be see that there's now no text content, but there is a tool call! It wants **Tavily Search tool** to be called.

This isn't calling that tool yet. In order to actually call it, the agent needs to be created

# Create the agent

Now that the tools and the LLM are defined, the agent can be created.  LangGraph will be used to construct the agent. Currently, a high level interface is used to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case the agent logic is to be modified.

Now, the agent can be initialized with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [31]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent
Now the agent can be run with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, as will be noted later how to get the outputs).

First up, let's see how it responds when there's no need to call a tool:


In [32]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================
Tool Calls:
  tavily_search (6ab7c928-5712-4a2f-a331-b38eecc90aaf)
 Call ID: 6ab7c928-5712-4a2f-a331-b38eecc90aaf
  Args:
    query: Hi!
================================= Tool Message =================================
Name: tavily_search

{"query": "Hi!", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.vocabulary.com/dictionary/hi", "title": "Hi - Definition, Meaning & Synonyms - Vocabulary.com", "content": "# hi The word *hi* is a good, nearly all-purpose greeting — it's appropriate in all but the most formal circumstances, so you might want to skip saying, \"*Hi*\" to the Queen when you meet her. When you greet someone, you have a lot of words to choose from, including \"hey,\" \"hello,\" \"hiya,\" and \"howdy,\" but *hi* is probably the most common. DISCLAIMER: These example 

In [34]:
input_message = {"role": "user", "content": "Search for the weather in London, United Kingdom"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Search for the weather in London, United Kingdom
================================== Ai Message ==================================
Tool Calls:
  tavily_search (067ed98a-b0c3-494e-a119-0b3443f17059)
 Call ID: 067ed98a-b0c3-494e-a119-0b3443f17059
  Args:
    query: weather in London, UK
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in London, UK", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in London, UK", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1756578116, 'localtime': '2025-08-30 19:21'}, 'current': {'last_updated_epoch': 1756577700, 'last_updated': '2025-08-30 19:15', 'temp_c': 17.2, 'temp_f': 63.0

# Streaming Messages

As seen earlier how the agent can be called with `.invoke` to get a final response. If the agent executes multiple steps, this may take a while. To show intermediate progress, we can stream back messages as they occur.

In [35]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in London, United Kingdom
================================== Ai Message ==================================
Tool Calls:
  tavily_search (c47c6bc6-8c08-4296-af9d-02213c96da84)
 Call ID: c47c6bc6-8c08-4296-af9d-02213c96da84
  Args:
    query: weather in London, UK
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in London, UK", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in London, UK", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.5171, 'lon': -0.1062, 'tz_id': 'Europe/London', 'localtime_epoch': 1756578116, 'localtime': '2025-08-30 19:21'}, 'current': {'last_updated_epoch': 1756577700, 'last_updated': '2025-08-30 19:15', 'temp_c': 17.2, 'temp_f': 63.0

# Streaming tokens

In addition to streaming back messages, it is also useful to stream back tokens. This can be done by specifying `stream_mode="messages"`

In [36]:
for step, metadata in agent_executor.stream({"messages": [input_message]}, stream_mode="messages"):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| London|,| United| Kingdom| is| light| rain| with| a| temperature| of| |17|.|2|°C| (|63|°F|).| The| wind| speed| is| |23| km|/h| (|14| mph|)| from| the| south|,| and| the| humidity| is| |94|%.|

# Adding in memory

As mentioned earlier, the agent is stateless. It does not remember previous interactions. To provide it memory, it needs to pass in a checkpointer. While this was passed in, `thread_id` is passed, when the agent is invoked (so the agent understands which thread to resume from)

In [37]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [38]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

In [39]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================
Tool Calls:
  tavily_search (f3cac22d-2827-435f-9bcf-42b0272af3f0)
 Call ID: f3cac22d-2827-435f-9bcf-42b0272af3f0
  Args:
    query: Hi, I
================================= Tool Message =================================
Name: tavily_search

{"query": "Hi, I", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://soundcloud.com/botwizzle", "title": "Hi, I'm Chris", "content": "Hi, I'm Chris, roughpatch, robin's ghost Fire Sale Posted 2 months ago 2 months ago Hip-Hop/Rap               Comment must not exceed 1000 characters    Hi, I'm Chris, roughpatch Operating System Posted 2 months ago 2 months ago Hip-Hop/Rap               Comment must not exceed 1000 characters    Hi, I'm Chris, roughpatch Data Mine Posted 2 months ago 2 months ago Hip-Hop/Rap               Comment must not e

In [40]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

{"name": "tavily_search", "parameters": {"query": "my name", "exclude_domains": ["twitter.com"], "include_favicon": True, "search_depth": "advanced"}}


In [41]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (62cf93a6-b5eb-483c-8f1d-43eb0ddadce9)
 Call ID: 62cf93a6-b5eb-483c-8f1d-43eb0ddadce9
  Args:
    query: my name
================================= Tool Message =================================
Name: tavily_search

{"query": "my name", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://es.wikipedia.org/wiki/Mi_nombre", "title": "Mi nombre - Wikipedia, la enciclopedia libre", "content": "_**Mi nombre**_ (en hangul: 마이 네임; RR: _Mai Neim_, en inglés, _**My Name**_, también conocida como _**Nemesis**_ o _**Undercover**_), es una serie de televisión surcoreana de suspense y acción estrenada el 15 de octubre de 2021 a través de Netflix.​​​ Después de esto, se anunció que el actor Ahn Bo-hyun ya se había realizado la prueba después de que un miembro del